>原文地址 https://www.cnblogs.com/green-crosswalk/p/8414134.html

gstreamer 使用不当，很容易引起内存泄漏。以下是注意事项：

1、gst_element_get_name

使用 g_free 释放 name，并设为 null

2、g_io_channel_read_line

g_free 释放 line，并设为 null

3、gst_structure_to_string/gst_caps_to_string

g_free 释放返回值，并设为 null

4、g_filename_to_uri

g_free 释放返回值，并设为 null

5、gst_pad_get_name/gst_omx_video_find_nearest_frame

不使用时，需要做 unref

6、gst_bus_add_watch

返回的 watchid 占用 source 资源，有两种释放方式：

1）callback 函数返回 FALSE，让 gstreamer 自动释放

2）主动调用 g_source_remove 释放；根据经验，只有当 handle message 函数收到 pipeline 切换到 NULL state 后，调用 g_source_remove 释放资源才有效

7、g_timeout_add

timer 资源通常采用 callback 函数返回 FALSE，让 gst 自动释放

8、gst_pad_get_peer

返回的 object，带有 ref，因此需要 gst_object_unref 释放

9、gst_buffer_map

此函数会对 buffer 做 ref，因此需要调用 gst_buffer_unmap

10、子类重载父类的 dispose，因此在子类调用结束时调用父类的 dispose 函数，G_OBJECT_CLASS(parent_class)->dispose(object)

11、g_object_get 获得 GstElement 类型变量时，会对返回的 element 做 ref 操作，因此 elment 不再使用时，调用 gst_object_unref